In [1]:
%pip install scikit-learn
%pip install transformers
%pip install torch


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch


Setting Up Vanna

In [3]:
from vanna.remote import VannaDefault
import os

In [4]:
api_key = os.getenv("VANNA_API_KEY")
vanna_model_name = 	"unimelbchatbottest"
vn = VannaDefault(model=vanna_model_name, api_key=api_key)
vn.connect_to_postgres(dbname="postgres",user="postgres.seqkcnapvgkwqbqipqqs",password="IT Web Server12",host="aws-0-ap-southeast-2.pooler.supabase.com",port=6543)

In [5]:
def run_query(question):
    query = vn.generate_sql(question, auto_train=False)
    data = vn.run_sql(query)
    answer = vn.generate_summary(question, data)
    return answer

Selecting Subjects

In [6]:
import json
import random

num_subjects = 1
subjects = []
num_chosen = 0

# Open and read the JSON file
with open('linkStorage/subjectLinks.json', 'r') as file:
    subject_names = json.load(file)
    # Generate random numbers which correspond to subjects
    while(num_chosen < num_subjects):
        rand_num = random.randrange(0, len(subject_names['links']))
        if(rand_num not in subjects):
            subjects.append(rand_num)
            num_chosen+=1
    # Switch out each of those numbers for (subject_code, subject_name) pairs
    for i in range(num_subjects):
        subjects[i] = (subject_names['links'][subjects[i]][0], subject_names['links'][subjects[i]][1])
print(subjects)

[('ABPL90434', 'Construction Management Internship')]


Selecting Majors

In [51]:
num_majors = 5
majors = []
num_chosen = 0

# Open and read the JSON file
with open('linkStorage/majorLinks.json', 'r') as file:
    major_names = json.load(file)
    # Generate random numbers which correspond to subjects
    while(num_chosen < num_majors):
        rand_num = random.randrange(0, len(major_names['links']))
        if(rand_num not in majors):
            majors.append(rand_num)
            num_chosen+=1
    # Switch out each of those numbers for (subject_code, subject_name) pairs
    for i in range(num_majors):
        majors[i] = major_names['links'][majors[i]][0]
print(majors)

['Cultural Studies', 'Plant Science', 'Agricultural Science', 'Land Systems', 'Water Resources']


Selecting Courses

In [69]:
num_courses = 5
courses = []
num_chosen = 0

# Open and read the JSON file
with open('linkStorage/courseLinks.json', 'r') as file:
    course_names = json.load(file)
    # Generate random numbers which correspond to subjects
    while(num_chosen < num_courses):
        rand_num = random.randrange(0, len(course_names['links']))
        if(rand_num not in courses):
            courses.append(rand_num)
            num_chosen+=1
    # Switch out each of those numbers for (subject_code, subject_name) pairs
    for i in range(num_courses):
        courses[i] = (course_names['links'][courses[i]][0], course_names['links'][courses[i]][1])
print(courses)

[('MC-DOPTOM', 'Doctor of Optometry'), ('MC-URBDES', 'Master of Urban Design'), ('MC-CTPYART', 'Master of Contemporary Art'), ('GC-CS', 'Graduate Certificate in Computer Science'), ('GC-TESOL', 'Graduate Certificate in TESOL')]


Generating Questions and Answers

In [7]:
subject_questions = [
    "Who is the subject coordinator of [subject]?",
    "What are the prerequisites for [subject]?",
    "What assessments does [subject] have?",
    "What is [subject]?",
    "When does [subject] run?",
]

subject_question_answers = []

# Takes subject (code, name) as input and adds 2 question answer pairs to subject_question_answers
def add_coordinator_answer(subject):
    with open(f"subjectInfo/{subject[0]}_info.json", 'r') as file:
        subject_data = json.load(file)
        answer = "The subject coordinator is "
        coordinators = []
        for i in range(len(subject_data["dates and times"])):
            for time in subject_data["dates and times"][i]:
                for title in subject_data["dates and times"][i][time][0]:
                    coordinator = subject_data["dates and times"][i][time][0][title]
                    if(not coordinators):
                        answer += coordinator
                        coordinators.append(coordinator)
                    elif(coordinator not in coordinators):
                        answer += f" and {coordinator}"
        subject_question_answers.append((f"Who is the subject coordinator of {subject[0]}?", answer))
        subject_question_answers.append((f"Who is the subject coordinator of {subject[1]}?", answer))

def add_prerequisite_answer(subject):
    with open(f"subjectInfo/{subject[0]}_info.json", 'r') as file:
        subject_data = json.load(file)
        answer = f"The prerequisites are {str(subject_data['pre-requisites'])}"
        subject_question_answers.append((f"What are the pre-requisites for {subject[0]}?", answer))
        subject_question_answers.append((f"What are the pre-requisites for {subject[1]}?", answer))

def add_assessment_answer(subject):
    with open(f"subjectInfo/{subject[0]}_info.json", 'r') as file:
        subject_data = json.load(file)
        answer = f"The assessments are {str(subject_data['assessments'])}"
        subject_question_answers.append((f"What are the assessments for {subject[0]}?", answer))
        subject_question_answers.append((f"What are the assessments for {subject[1]}?", answer))

def add_description_answer(subject):
    with open(f"subjectInfo/{subject[0]}_info.json", 'r') as file:
        subject_data = json.load(file)
        answer = f"{(subject_data["overview"])} {str(subject_data["aims"])} {str(subject_data["indicative content"])}"
        subject_question_answers.append((f"What is {subject[0]}?", answer))
        subject_question_answers.append((f"What is {subject[1]}?", answer))

def add_timing_answer(subject):
    with open(f"subjectInfo/{subject[0]}_info.json", 'r') as file:
        subject_data = json.load(file)
        answer = f"{subject[0]}, {subject[1]} runs in "
        times = []
        for i in range(len(subject_data['dates and times'])):
            for time in subject_data['dates and times'][i].keys():
                if(not times):
                    answer += time
                    times.append(time)
                elif(not time in times):
                    answer += f" and {time}"
        subject_question_answers.append((f"When does {subject[0]} run?", answer))
        subject_question_answers.append((f"When does {subject[1]} run?", answer))


for subject in subjects:
    add_coordinator_answer(subject)
    #add_prerequisite_answer(subject) This one is very difficult due to formatting
    add_assessment_answer(subject)
    add_description_answer(subject)
    add_timing_answer(subject)

for pair in subject_question_answers:
    print(pair)


('Who is the subject coordinator of ABPL90434?', 'The subject coordinator is Paulo Vaz Serra')
('Who is the subject coordinator of Construction Management Internship?', 'The subject coordinator is Paulo Vaz Serra')
('What are the assessments for ABPL90434?', "The assessments are [{'description': 'Successful completion of 160 hours industry placement.. 160 hours. Hurdle requirement:. Placement must  be completed', 'timing': 'Throughout the teaching period', 'percentage': 'N/A'}, {'description': 'A report outlining the structure, policies and activities of the host organisation and the case study to be reported on.. 2000 words. Hurdle requirement:. Submission is hurdle  requirement .', 'timing': 'Within two weeks of beginning the internship', 'percentage': '20%'}, {'description': 'Journal that documents and reflects on the internship experience, an hours log and a host supervisor assessment report.. 2000 words. Hurdle requirement:. Submission is  hurdle .', 'timing': 'Within three weeks 

In [52]:
major_question_answers = []

def add_major_def_answer(major):
    with open(f"majorInfo/{major}_info.json", 'r') as file:
        major_data = json.load(file)
        answer = str(major_data['overview']) + str(major_data['structure']['Structure'])
    major_question_answers.append((f"What is the major {major}?", answer))

def add_major_subjects_answer(major):
    with open(f"majorInfo/{major}_info.json", 'r') as file:
        major_data = json.load(file)
        answer = str(major_data['structure']['Structure'])
    major_question_answers.append((f"What subjects do I need to complete {major}?", answer))

for major in majors:
    add_major_def_answer(major)
    add_major_subjects_answer(major)

for pair in major_question_answers:
    print(pair)

('What is the major Cultural Studies?', "['See program overview']['50 credit points', 'Duration:6 months full-time / 1 year part-time', 'Program Requirements -', ['elective subjects (50 points)'], 'Total 50 points']")
('What subjects do I need to complete Cultural Studies?', "['50 credit points', 'Duration:6 months full-time / 1 year part-time', 'Program Requirements -', ['elective subjects (50 points)'], 'Total 50 points']")
('What is the major Plant Science?', "['A Plant Science major will provide the springboard for students in entering careers or research in all fields requiring a solid understanding of plants and their environments. Graduates will develop a comprehensive integrated knowledge of the biology of plants as well as both field and laboratory skills. This major will include knowledge from all aspects of plant biology from cells to ecosystems, by enabling students to complete an integrated subject on plant evolution or an advanced lab and coursework subject in plant biote

In [70]:
course_question_answers = []

def add_course_def_answer(course):
    with open(f"courseInfo/{(course[1]).replace(" ", "")}_info.json", 'r') as file:
        course_data = json.load(file)
        answer = str(course_data['Overview text']) + str(course_data['Overview table'])
    course_question_answers.append((f"What is the course {course[0]}?", answer))
    course_question_answers.append((f"What is the course {course[1]}?", answer))

for course in courses:
    add_course_def_answer(course)

for pair in course_question_answers:
    print(pair)

('What is the course MC-DOPTOM?', "['Optometrists are independent, primary health care professionals trained to examine the eyes and visual system. Optometrists diagnose, treat and manage diseases and injuries of the eye and associated structures. Optometrists are experts in the management of vision disorders, and in the correction of reduced vision. Numerous systemic conditions affect vision and ocular structures, so optometrists also play an important role in identifying general health disorders. Optometrists prescribe spectacles, contact lenses, medications for ocular conditions, vision therapy, and low vision rehabilitation.', 'The Doctor of Optometry (OD) is a masters-level professional degree. The OD program is the first of its kind in the southern hemisphere, creating a new benchmark in optometric education. It is an internationally recognised qualification with full accreditation from the Optometry Council of Australia and New Zealand. Completion of the OD satisfies the legisla

Using BERT to test

In [8]:
# This was found to be ineffective
"""
def get_bert_embeddings(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # We use the embeddings from the [CLS] token (first token)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

def bert_cosine_similarity(paragraph, summary):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    
    para_embedding = get_bert_embeddings(paragraph, model, tokenizer)
    summary_embedding = get_bert_embeddings(summary, model, tokenizer)
    
    # Cosine similarity
    return cosine_similarity([para_embedding], [summary_embedding])[0][0]
"""

In [10]:
"""
threshold = 0.78

for question_answer_pair in subject_question_answers:
    paragraph = question_answer_pair[1] # The actual data
    summary = run_query(question_answer_pair[0]) # AI response to the question
    similarity = bert_cosine_similarity(paragraph, summary)
    print(similarity)
    if(similarity<threshold):
        raise Exception(f"Answer found that does not meet standard.\nParagraph:\n{paragraph}\nLLM Response:\n{summary}")
"""

SQL Prompt: [{'role': 'system', 'content': 'You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE  TABLE "public".courses ( \n\taqf_level            varchar(100)    ,\n\tcredit_points        varchar(150)    ,\n\tcrisco               varchar(10)    ,\n\tduration             text    ,\n\tfees_information     text    ,\n\tlocation             varchar(15)    ,\n\tstart_year           smallint    ,\n\tstudy_level          text    ,\n\tattributes_outcomes_skills text    ,\n\tindicative_content   text[]    ,\n\tcourse_code          varchar(10)  NOT NULL  ,\n\tcourse_name          text    ,\n\tcourse_structure     json    ,\n\tinherent_requirements json    ,\n\tentry_requirements   json    ,\n\tCONSTRAINT pk_course PRIMARY KEY ( course_code ),\n\tCONSTRAINT unq_courses_course_name UNIQUE ( course_name ) \n );\n\nCREATE  TABL

ValidationError: syntax error at or near "The"
LINE 1: The provided context does not contain information about a su...
        ^


Testing With Cosine Similiarity (No BERT)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
def cosine_sim(paragraph, summary):
    # Combine both texts into a list
    texts = [paragraph, summary]
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    # Transform texts into vectors
    vectors = vectorizer.fit_transform(texts)
    # Compute cosine similarity
    cosine_sim_value = cosine_similarity(vectors[0:1], vectors[1:2])
    return cosine_sim_value[0][0]

In [54]:
import warnings

# Testing TF-IDF + Cosine Similarity on subjects

threshold = 0.2

for question_answer_pair in subject_question_answers:
    paragraph = question_answer_pair[1] # The actual data
    summary = run_query(question_answer_pair[0]) # AI response to the question
    similarity = cosine_sim(paragraph, summary)
    print(similarity)
    if(similarity<threshold):
        warnings.warn(f"Answer found that does not meet standard.\nParagraph:\n{paragraph}\nLLM Response:\n{summary}")



SQL Prompt: [{'role': 'system', 'content': 'You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE  TABLE "public".courses ( \n\taqf_level            varchar(100)    ,\n\tcredit_points        varchar(150)    ,\n\tcrisco               varchar(10)    ,\n\tduration             text    ,\n\tfees_information     text    ,\n\tlocation             varchar(15)    ,\n\tstart_year           smallint    ,\n\tstudy_level          text    ,\n\tattributes_outcomes_skills text    ,\n\tindicative_content   text[]    ,\n\tcourse_code          varchar(10)  NOT NULL  ,\n\tcourse_name          text    ,\n\tcourse_structure     json    ,\n\tinherent_requirements json    ,\n\tentry_requirements   json    ,\n\tCONSTRAINT pk_course PRIMARY KEY ( course_code ),\n\tCONSTRAINT unq_courses_course_name UNIQUE ( course_name ) \n );\n\nCREATE  TABL

KeyboardInterrupt: 

In [ ]:
# Testing TF-IDF + Cosine Similarity on majors

threshold = 0.35

for question_answer_pair in major_question_answers:
    paragraph = question_answer_pair[1] # The actual data
    summary = run_query(question_answer_pair[0]) # AI response to the question
    print(summary)
    similarity = cosine_sim(paragraph, summary)
    print(similarity)
    if(similarity<threshold):
        warnings.warn(f"Answer found that does not meet standard.\nParagraph:\n{paragraph}\nLLM Response:\n{summary}")

In [71]:
# Testing TF-IDF + Cosine Similarity on courses

threshold = 0.2

for question_answer_pair in course_question_answers:
    paragraph = question_answer_pair[1] # The actual data
    summary = run_query(question_answer_pair[0]) # AI response to the question
    print(summary)
    similarity = cosine_sim(paragraph, summary)
    print(similarity)
    if(similarity<threshold):
        warnings.warn(f"Answer found that does not meet standard.\nParagraph:\n{paragraph}\nLLM Response:\n{summary}")

SQL Prompt: [{'role': 'system', 'content': 'You are a PostgreSQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE  TABLE "public".courses ( \n\taqf_level            varchar(100)    ,\n\tcredit_points        varchar(150)    ,\n\tcrisco               varchar(10)    ,\n\tduration             text    ,\n\tfees_information     text    ,\n\tlocation             varchar(15)    ,\n\tstart_year           smallint    ,\n\tstudy_level          text    ,\n\tattributes_outcomes_skills text    ,\n\tindicative_content   text[]    ,\n\tcourse_code          varchar(10)  NOT NULL  ,\n\tcourse_name          text    ,\n\tcourse_structure     json    ,\n\tinherent_requirements json    ,\n\tentry_requirements   json    ,\n\tCONSTRAINT pk_course PRIMARY KEY ( course_code ),\n\tCONSTRAINT unq_courses_course_name UNIQUE ( course_name ) \n );\n\nCREATE  TABL